In [1]:
# https://github.com/computationalpathologygroup/ASAP/releases에서 실행 파일을 다운받고, 바이너리 폴더를 path에 append 해줍니다.
import sys
sys.path.append('C:\\Program Files\\ASAP 2.2\\bin')

# bin 폴더의 multiresolutionimageinterface.py 파일을 import 해줍니다.
import multiresolutionimageinterface as mir
import cv2
from tqdm import tqdm_notebook
import os



In [2]:
# tif 파일, xml 형식의 annotation 파일 불러오기 위함
reader = mir.MultiResolutionImageReader()
annotation_list = mir.AnnotationList()
xml_repository = mir.XmlRepository(annotation_list)

In [3]:
# 각각 annotation파일, training용 wsi파일 주소
dirAnnotations = '../data/annotations/'
dirData = '../data/training/'

In [4]:
# '../data/training/center_0\\patient_001_node_1.tif' 형식의 파일명만 리스트에 저장
ImageFiles = []
# r=root, d=directories, f = files
for r, d, f in os.walk(dirData):
    for file in f:
        if '.tif' in file and 'mask' not in file:
            ImageFiles.append(os.path.join(r, file))

In [5]:
# 전체 이미지였다면 500개의 이미지 파일명 저장
ImageFiles

['../data/training/center_0\\patient_001_node_1.tif',
 '../data/training/center_0\\patient_004_node_0.tif',
 '../data/training/center_0\\patient_004_node_1.tif',
 '../data/training/center_0\\patient_004_node_2.tif',
 '../data/training/center_0\\patient_004_node_3.tif',
 '../data/training/center_0\\patient_004_node_4.tif']

In [11]:
# xml파일들이 위치한 폴더를 받아서 각 annotation파일을 마스크로 변환(종양인 부위는 흰색, 없는 부위는 검은색)
def CreateAnnotationMask(annotationPath):
    
    # xml파일의 디렉토리와 확장자 빼고 이름만 저장 ex) patient_001_node_1
    fileNamePart = annotationPath.replace('.xml','').replace(dirAnnotations, "")
    
    # fileNamePart에 .tif 확장자 부여 ex) patient_001_node_1.tif
    tifName = fileNamePart + '.tif'

    # tifName과 ImageFiles 내 값 중 일치하는 것이 없다면 skip -> annotation이 존재하는(tumor가 존재하는) 이미지에 대해서만 실행되는 것.
    partialMatches = [s for s in ImageFiles if tifName in s]
    if len(partialMatches) == 0:
        print('Warning - This file is missing from the file list: {0} - skipping.'.format(tifName))
        return
    tifPath = partialMatches[0]
    
    # tif 파일이 없다면 skip
    if (not os.path.isfile(tifPath)): 
        print('Warning - Could not locate {0} - skipping this annotation file.'.format(tifPath))
        return
    
    # 이미 존재하는 파일이면 skip
    maskPath = tifPath.replace('.tif', '_mask.tif')
    if (os.path.isfile(maskPath)):
        print('Info - Mask file of {0} already exists - skipping'.format(tifPath))
        return
    
    # xml 파일 받아오기
    xml_repository.setSource(annotationPath)
    xml_repository.load()

    # 폴리곤으로 존재하는 xml파일을 마스크 tif 파일로 변환
    annotation_mask = mir.AnnotationToMask()
    mr_image = reader.open(tifPath)
    if(mr_image is None):
        print('Warning - Could not read {0} - skipping'.format(tifPath))
        return
    label_map = {'metastases': 1, 'normal': 2}
    conversion_order = ['metastases', 'normal']
    annotation_mask.convert(annotation_list, 
                            maskPath, 
                            mr_image.getDimensions(), 
                            mr_image.getSpacing(), 
                            label_map, 
                            conversion_order)

In [12]:
# annotation 파일 명 저장하는 리스트 생성
AnnotationFiles = []
# r=root, d=directories, f = files
for r, d, f in os.walk(dirAnnotations):
    for file in f:
        if '.xml' in file:
            AnnotationFiles.append(os.path.join(r, file))

In [13]:
AnnotationFiles[:4]

['../data/annotations/patient_004_node_4.xml',
 '../data/annotations/patient_009_node_1.xml',
 '../data/annotations/patient_010_node_4.xml',
 '../data/annotations/patient_012_node_0.xml']

In [14]:
from tqdm.notebook import tqdm

# 위에서 선언한 함수로 annotation이 존재하는 tif파일 ex) patient_004_node_4.tif 파일 위치에 patient_004_node_4_mask.tif 파일 생성
for f in tqdm(AnnotationFiles, 'Creating masks...'):
    print('Annotation file: ' + f)
    CreateAnnotationMask(f)


Creating masks...:   0%|          | 0/50 [00:00<?, ?it/s]

Annotation file: ../data/annotations/patient_004_node_4.xml
Info - Mask file of ../data/training/center_0\patient_004_node_4.tif already exists - skipping
Annotation file: ../data/annotations/patient_009_node_1.xml
Warning - This file is missing from the file list: patient_009_node_1.tif - skipping.
Annotation file: ../data/annotations/patient_010_node_4.xml
Warning - This file is missing from the file list: patient_010_node_4.tif - skipping.
Annotation file: ../data/annotations/patient_012_node_0.xml
Warning - This file is missing from the file list: patient_012_node_0.tif - skipping.
Annotation file: ../data/annotations/patient_015_node_1.xml
Warning - This file is missing from the file list: patient_015_node_1.tif - skipping.
Annotation file: ../data/annotations/patient_015_node_2.xml
Warning - This file is missing from the file list: patient_015_node_2.tif - skipping.
Annotation file: ../data/annotations/patient_016_node_1.xml
Warning - This file is missing from the file list: pati